# Iris Classification Pipeline with ZenML

This notebook demonstrates a ZenML pipeline for iris classification, including data loading, model training, evaluation, explainability, and data drift detection.

In [ ]:
!zenml connect --url=https://d13d987c-zenml.cloudinfra.zenml.io

In [32]:
from zenml.client import Client

In [44]:
Client().activate_stack("default_with_s3")

In [54]:
!zenml stack describe 'local-aws-step-operator'
Client().activate_stack("local-aws-step-operator")

⠋ Describing the stack...
              Stack Configuration               
┏━━━━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ COMPONENT_TYPE     │ COMPONENT_NAME          ┃
┠────────────────────┼─────────────────────────┨
┃ ORCHESTRATOR       │ default                 ┃
┠────────────────────┼─────────────────────────┨
┃ STEP_OPERATOR      │ aws-sagemaker-pipelines ┃
┠────────────────────┼─────────────────────────┨
┃ ARTIFACT_STORE     │ aws-sagemaker-pipelines ┃
┠────────────────────┼─────────────────────────┨
┃ CONTAINER_REGISTRY │ aws-sagemaker-pipelines ┃
┠────────────────────┼─────────────────────────┨
┃ IMAGE_BUILDER      │ aws-sagemaker-pipelines ┃
┗━━━━━━━━━━━━━━━━━━━━┷━━━━━━━━━━━━━━━━━━━━━━━━━┛
        'local-aws-step-operator' stack         
⠙ Describing the stack...
           Labels           
┏━━━━━━━━━━━━━━━━━━┯━━━━━━━┓
┃ LABEL            │ VALUE ┃
┠──────────────────┼───────┨
┃ zenml:full_stack │ True  ┃
┗━━━━━━━━━━━━━━━━━━┷━━━━━━━┛
⠙ Describing the stack...
Stack 'local-

In [46]:
# !zenml stack describe 'aws-sagemaker-pipelines'
# Client().activate_stack('aws-sagemaker-pipelines')

In [47]:
from typing import Any, Dict, Tuple

import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from typing_extensions import Annotated
from zenml import log_artifact_metadata, step


def safe_metadata(data: Any) -> Dict[str, Any]:
    """Create metadata dict with only supported types."""
    metadata = {"shape": data.shape}
    if isinstance(data, pd.DataFrame):
        metadata["columns"] = list(data.columns)
    return metadata


@step
def load_data() -> (
    Tuple[
        Annotated[pd.DataFrame, "X_train"],
        Annotated[pd.DataFrame, "X_test"],
        Annotated[pd.Series, "y_train"],
        Annotated[pd.Series, "y_test"],
    ]
):
    """Load the iris dataset and split into train and test sets."""
    iris = load_iris(as_frame=True)
    X = iris.data
    y = iris.target
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    for name, data in [
        ("X_train", X_train),
        ("X_test", X_test),
        ("y_train", y_train),
        ("y_test", y_test),
    ]:
        log_artifact_metadata(
            artifact_name=name, metadata={"dataset_info": safe_metadata(data)}
        )

    return X_train, X_test, y_train, y_test

In [55]:
import pandas as pd
from sklearn.svm import SVC
from typing_extensions import Annotated
from zenml import (
    ArtifactConfig,
    log_artifact_metadata,
    log_model_metadata,
    step,
)
from zenml.integrations.aws.flavors.sagemaker_step_operator_flavor import (
    SagemakerStepOperatorSettings,
)


@step(
    enable_cache=False,
    step_operator="aws-sagemaker-pipelines",
    settings={
        "step_operator.sagemaker": SagemakerStepOperatorSettings(
            estimator_args={"instance_type": "ml.p3.2xlarge"}
        )
    },
)
def train_model(
    X_train: pd.DataFrame,
    y_train: pd.Series,
) -> Annotated[SVC, ArtifactConfig(name="model", is_model_artifact=True)]:
    """Train an SVM classifier."""
    model = SVC(kernel="rbf", probability=True)
    model.fit(X_train, y_train)
    train_accuracy = model.score(X_train, y_train)

    log_model_metadata(
        metadata={
            "training_metrics": {
                "train_accuracy": float(train_accuracy),
            },
            "model_info": {
                "model_type": type(model).__name__,
                "kernel": model.kernel,
            },
        }
    )

    log_artifact_metadata(
        artifact_name="model",
        metadata={
            "model_details": {
                "type": type(model).__name__,
                "kernel": model.kernel,
                "n_support": model.n_support_.tolist(),
            }
        },
    )

    return model

In [56]:
from typing import Tuple

import numpy as np
import pandas as pd
from sklearn.svm import SVC
from typing_extensions import Annotated
from zenml import log_artifact_metadata, log_model_metadata, step


@step
def evaluate_model(
    model: SVC,
    X_test: pd.DataFrame,
    y_test: pd.Series,
) -> Tuple[
    Annotated[np.ndarray, "predictions"],
    Annotated[np.ndarray, "probabilities"],
]:
    """Evaluate the model and make predictions."""
    test_accuracy = model.score(X_test, y_test)
    predictions = model.predict(X_test)
    probabilities = model.predict_proba(X_test)

    log_model_metadata(
        metadata={
            "evaluation_metrics": {
                "test_accuracy": float(test_accuracy),
            }
        }
    )

    log_artifact_metadata(
        artifact_name="predictions",
        metadata={
            "prediction_info": {
                "shape": predictions.shape,
                "unique_values": np.unique(predictions).tolist(),
            }
        },
    )

    log_artifact_metadata(
        artifact_name="probabilities",
        metadata={
            "probability_info": {
                "shape": probabilities.shape,
                "min": float(np.min(probabilities)),
                "max": float(np.max(probabilities)),
            }
        },
    )

    return predictions, probabilities

In [61]:
from typing import Dict

import pandas as pd
import shap
from sklearn.svm import SVC
from typing_extensions import Annotated
from zenml import log_artifact_metadata, step


class SHAPVisualization:
    def __init__(self, shap_values, feature_names):
        self.shap_values = shap_values
        self.feature_names = feature_names


@step
def explain_model(
    model: SVC, X_train: pd.DataFrame
) -> Annotated[SHAPVisualization, "shap_visualization"]:
    """Generate SHAP values for model explainability and create a visualization."""
    explainer = shap.KernelExplainer(
        model.predict_proba, shap.sample(X_train, 100)
    )
    shap_values = explainer.shap_values(X_train.iloc[:100])

    log_artifact_metadata(
        artifact_name="shap_values",
        metadata={
            "shap_info": {
                "shape": [arr.shape for arr in shap_values],
                "n_classes": len(shap_values),
                "n_features": shap_values[0].shape[1],
            }
        },
    )

    return SHAPVisualization(shap_values, X_train.columns)

In [62]:
from typing import Dict

import pandas as pd
from scipy.stats import ks_2samp
from typing_extensions import Annotated
from zenml import log_artifact_metadata, step


@step
def detect_data_drift(
    X_train: pd.DataFrame,
    X_test: pd.DataFrame,
) -> Annotated[Dict[str, float], "drift_metrics"]:
    """Detect data drift between training and test sets."""
    drift_metrics = {}
    for column in X_train.columns:
        _, p_value = ks_2samp(X_train[column], X_test[column])
        drift_metrics[column] = p_value

    log_artifact_metadata(
        artifact_name="drift_metrics",
        metadata={
            "drift_summary": {
                "high_drift_features": [
                    col for col, p in drift_metrics.items() if p < 0.05
                ]
            }
        },
    )

    return drift_metrics

In [63]:
from zenml import Model, pipeline


@pipeline(
    settings={
        # "docker": DockerSettings(python_package_installer="uv",
        # requirements="requirements.txt"),
        # "resources": ResourceSettings(memory="8GB"),
    },
    model=Model(name="high_risk_classification"),
)
def iris_classification_pipeline():
    X_train, X_test, y_train, y_test = load_data()
    model = train_model(X_train, y_train)
    evaluate_model(model, X_test, y_test)
    explain_model(model, X_train)
    drift_metrics = detect_data_drift(X_train, X_test)

In [64]:
# Run the pipeline
pipeline_run = iris_classification_pipeline()

Initiating a new run for the pipeline: iris_classification_pipeline.
Archiving notebook code...
Code already exists in artifact store, skipping upload.
